# Declare libraries

In [1]:
from osgeo import gdal
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt
import numpy as np
import read_cover_fraction as rcf
from mpl_toolkits.basemap import Basemap
import numpy as np
import shapefile as shp
from pyproj import Proj

# Declare functions

## function to read the data

In [2]:
def get_dataSG(fname,latmax,latmin,lonmax,lonmin):

    gdal.UseExceptions()
    ds = gdal.Open(fname)
    data = ds.ReadAsArray()
    gt = ds.GetGeoTransform()
    #
    #
    #
    xres = gt[1]
    yres = gt[5]
    #
    xmin = gt[0]
    ymin = gt[3]
    #
    xmax = gt[0] + (xres * ds.RasterXSize)
    ymax = gt[3] + (yres * ds.RasterYSize)


    X=np.arange(xmin+xres,xmax+xres,xres)
    Y=np.arange(ymin+yres,ymax+yres,yres)


    #Chunck the data to save RAM for plotting
    #Ydecreasing
    b=np.min(np.where(Y<latmin)[0])
    a=np.max(np.where(Y>latmax)[0])

    c=np.max(np.where(X<lonmin)[0])
    d=np.min(np.where(X>lonmax)[0])

#    print(np.max(data))
#    print(np.min(data))
    #ECOCLIMAP data have no projections so no need to reproject the data
    #to be verified for other datasets
    LAT=Y[a:b]
    LON=X[c:d]
    D=data[a:b,c:d]

    return (LAT,LON,D)

## Read station coordinates function

In [3]:
def readstationcoordinate(fnstation):
    data=open(fnstation,'r')
    lat=[]
    lon=[]
    for d in data:
        if 'Latitude' not in d:
            c=d.strip(' ').split(' ')
            lat.append(c[0])
            lon.append(c[len(c)-1])
    LAT=np.array(lat).astype(float)
    LON=np.array(lon).astype(float)
    return(LAT,LON)

## Function to set correspondance between coastline and line in the csv files

In [4]:
def set_covernumber_station(LAT,LON,LATSG,LONSG,covernum,DSG):
    Linenum=[]
    indicelat=[]
    indicelon=[]
    for x,_ in enumerate(LAT):
        idx=np.argmin(np.abs(LAT[x]-LATSG))
        idy=np.argmin(np.abs(LON[x]-LONSG))
        Linenum.append(np.where(covernum==int(DSG[idx,idy]))[0][0])
        indicelat.append(idx)
        indicelon.append(idy)
    return (Linenum,indicelon,indicelat)

## Write cover station text file

In [5]:
def write_txt_file_cover_station(fntxt,LAT,LON,Linenum,covernum,covername,cityfrac,vegfrac,inwaterfrac,seafrac):
    data=open(fntxt,'w')
    data.write('LAT, LON, Covernum, Covername, City fraction, Vegetation fraction, Inland water fraction, sea fraction\n')
    for x,_ in enumerate(LAT):
        data.write(str(LAT[x])+', '+str(LON[x])+', '+str(covernum[Linenum[x]])+', '+covername[Linenum[x]]+', '+
                   str(cityfrac[Linenum[x]])+', '+str(vegfrac[Linenum[x]])+', '+
                   str(inwaterfrac[Linenum[x]])+', '+str(seafrac[Linenum[x]])+'\n')
    data.close()
    return

# Function to calculate distance between LAT, LON points

In [6]:
def distance_latlon(lat2,lon2,latref,lonref):
    R = 6373.0 *1000 # radius of the Earth

    dlon = np.deg2rad(lon2) - np.deg2rad(lonref) #change in coordinates

    dlat = np.deg2rad(lat2) - np.deg2rad(latref)

    a = np.sin(dlat / 2)**2 + np.cos(np.deg2rad(latref)) * np.cos(np.deg2rad(lat2)) * np.sin(dlon / 2)**2 #Haversine formula

    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return (distance)   

# MAIN 

### Open river data

In [36]:
import shapefile as shp

def read_shapefile(sf):
    #fetching the headings from the shape file
    fields = [x[0] for x in sf.fields][1:]

#fetching the records from the shape file
    records = [list(i) for i in sf.records()]
    shps = [s.points for s in sf.shapes()]
    names = [r[3] for r in records]
    asso_unit = [r[1] for r in records]

##converting shapefile data into pandas dataframe
#    df = pd.DataFrame(columns=fields, data=records)
#
##assigning the coordinates
#    df = df.assign(coords=shps)
    return fields,records,shps, names, asso_unit

In [37]:
from osgeo import gdal,ogr
daShapefile='/mnt/g/RiversandLakes_06022020/Rivers&Lakes/Shapefiles/WATER_RivNetRoutes.shp'
driverName = "ESRI Shapefile"
drv=ogr.GetDriverByName( driverName )
dataSource = driver.Open(daShapefile, 0)
layer = dataSource.GetLayer()


#read the shapefile
sf=shp.Reader(daShapefile)    
fields,records,shps, names,asso_unit = read_shapefile(sf)
indices=np.arange(0,len(names)-1)

In [ ]:

    


def plt_map_fill(idi,sf,cmapname,figname,sizefig,names) :
    Names=np.array(names)
    Nu=np.unique(Names)    
    
    
    fig = plt.figure(figsize=sizefig)
    ax1=fig.gca()
    m = Basemap(projection='tmerc', lon_0=-8, lat_0=53.5,llcrnrlon=-11.7,llcrnrlat=51.1,
                urcrnrlat=55.5,urcrnrlon=-5,resolution='f',ax=ax1)
    m.drawcountries()
    m.drawcoastlines(linewidth=.5)
    m.drawmeridians(np.arange(-10,-6,2),color='k', linewidth=1.0)
    m.drawparallels(np.arange(53,56,1),color='k', linewidth=1.0)
    colorsmap=plt.get_cmap(cmapname,len(Nu)) 
    
    for j,i in enumerate(idi) :
        shape_ex = sf.shape(i)
        x_lon = np.zeros((len(shape_ex.points),1))
        y_lat = np.zeros((len(shape_ex.points),1))
        for ip in range(len(shape_ex.points)):
            x_lon[ip] = shape_ex.points[ip][0]
            y_lat[ip] = shape_ex.points[ip][1]
        lons,lats=pnyc(x_lon,y_lat,inverse=True)
        X,Y=m(lons,lats)
        plt.fill(X,Y,color=colorsmap(j))
    plt.legend(P,names)
    plt.savefig(figname)    
    return
    
#fn='/home/gbessardon/CORINE2018/CLC18_IE_ITM/CLC18_IE_ITM.shp'
fn='/home/gbessardon/national_soils/SOIL_SISNationalSoils_shp/Data/SOIL_SISNationalSoils_Shp/SOIL_SISNationalSoils.shp'
#read the shapefile
sf=shp.Reader(fn)    
fields,records,shps, names,asso_unit = read_shapefile(sf)
indices=np.arange(0,len(names)-1)
#generates a function to convert the prjected data (needs a function to read it in the Readme.txt)
pnyc = Proj(proj='tmerc',ellps='GRS80',lat_0=53.5,lon_0=-8, x_0=200000, y_0=250000, k_0=1.000035)
#+a 	Semimajor radius of the ellipsoid axis
#+axis 	Axis orientation
#+b 	Semiminor radius of the ellipsoid axis
#+ellps 	Ellipsoid name (see proj -le)
#+k 	Scaling factor (deprecated)
#+k_0 	Scaling factor
#+lat_0 	Latitude of origin
#+lon_0 	Central meridian
#+lon_wrap 	Center longitude to use for wrapping (see below)
#+over 	Allow longitude output outside -180 to 180 range, disables wrapping (see below)
#+pm 	Alternate prime meridian (typically a city name, see below)
#+proj 	Projection name (see proj -l)
#+units 	meters, US survey feet, etc.
#+vunits 	vertical units.
#+x_0 	False easting
#+y_0 	False northing
Names=np.array(names)
Nu=np.unique(Names)
colorsmap=plt.get_cmap('terrain_r',len(Nu))
fig = plt.figure(figsize=(40,60))
ax1=fig.gca()
m = Basemap(projection='tmerc', lon_0=-8, lat_0=53.5,llcrnrlon=-11.7,llcrnrlat=51.1,
            urcrnrlat=55.5,urcrnrlon=-5,resolution='f',ax=ax1)
m.drawcountries()
m.drawcoastlines(linewidth=.5)
m.drawmeridians(np.arange(-10,-6,2),color='k', linewidth=1.0)
m.drawparallels(np.arange(53,56,1),color='k', linewidth=1.0)
P=np.zeros(len(Nu))
for nb,n in enumerate(Nu):
    idi=np.where(Names==Nu[0])[0]
    for j,i in enumerate(idi) :
        shape_ex = sf.shape(i)
        x_lon = np.zeros((len(shape_ex.points),1))
        y_lat = np.zeros((len(shape_ex.points),1))
        for ip in range(len(shape_ex.points)):
            x_lon[ip] = shape_ex.points[ip][0]
            y_lat[ip] = shape_ex.points[ip][1]
        

In [21]:
L=dataSource.GetLayer()
spatialRef = L.GetSpatialRef()
extent=L.GetExtent()


In [26]:
F=L.GetFeature(0)

In [31]:
F.GetField(0)

350347

In [9]:
spatialRef.ExportToProj4()

'+proj=tmerc +lat_0=53.5 +lon_0=-8 +k=1.000035 +x_0=200000 +y_0=250000 +a=6377340.189 +rf=299.3249646 +towgs84=482.5,-130.6,564.6,-1.042,-0.214,-0.631,8.15 +units=m +no_defs'

In [10]:
Fe=L.GetFeature

In [32]:
from osgeo import gdal

In [33]:
gdal.Open(f)

## Open ECOCLIMAP-SG data

In [ ]:
latmax=55.5
latmin=50.9
lonmax=-5.4
lonmin=-11

fnameSG='/mnt/g/ECOCLIMAP/ECOCLIMAPSG/ecosg_final_map.dir'
(LATSG,LONSG,DSG)=get_dataSG(fnameSG,latmax,latmin,lonmax,lonmin)
covernamef='/mnt/g/ECOCLIMAP/ECOCLIMAPSG/ECOCLIMAP_SG_cover_data.csv'
#get the cover deatils from the csvfile
(fieldnames,covernum,covername,cityfrac,vegfrac,inwaterfrac,seafrac)=rcf.cover_fraction(covernamef)

# Create ECOCLIMAP-SG colormap

In [ ]:
a0=(0/255.0,0/255.0,0/255.0,255/255.0)## 0 value
a=(0/255.0,0/255.0,128/255.0,255/255.0)##navy blue 1.sea    
b=(0/255.0,0/255.0,205/255.0,255/255.0)## mediumblue 2.inland waters
c=(0/255.0, 0/255.0, 255/255.0, 255/255.0) ##Blue 3.rivers
d=(211/255.0,211/255.0,211/255.0,255/255.0)## lightgray 4.Bare land
e=(169/255.0,169/255.0,169/255.0,255/255.0)## darkgray 5.Rocks
f=(255/255.0,250/255.0,250/255.0,255/255.0) ## Snow 6.permanent snow
g=(240/255.0,255/255.0,240/255.0,255/255.0)## Honeydew 7.boreal broadleaf deciduous
h=(85/255.0,107/255.0,47/255.0,255/255.0)## darkolivegreen 8.temperate broadleaf deciduous
ii=(154/255.0,205/255.0,50/255.0,255/255.0)## yellowgreen 9.tropical broadleaf deciduous
k=(0/255.0,128/255.0,0/255.0,255/255.0)## green 10.temperate broadleaf evergreen
l=(255/255.0,127/255.0,80/255.0,255/255.0)## coral 11. tropical broadleaf evergreen
m=(160/255.0,82/255.0,45/255.0,255/255.0)## siena 12. boreal needleaf evergreen
n=(34/255.0,139/255.0,34/255.0,255/255.0)## forest green 13.temperate needleleaf evergreen
o= (188/255.0,143/255.0,143/255.0,255/255.0)## rosybrown 14. boreal needleleaf deciduous
p=(205/255.0,133/255.0,63/255.0,255/255.0)## peru 15. shrubs
q=(222/255.0,184/255.0,135/255.0,255/255.0)##  burlywood 16. boreal grassland
r=(50/255.0,205/255.0,50/255.0,255/255.0)##limegreen 17 . temperate grassland
s=(255/255.0,215/255.0,0/255.0,255/255.0) ##gold 18. tropical grassland
t=(32/255.0,178/255.0,170/255.0,255/255.0)##lightseagreen 19.winter crop
u=(173/255.0,255/255.0,47/255.0,255/255.0)##green yellow 20.summer crop
v=(189/255.0,183/255.0,107/255.0,255/255.0)##darkkhaki 21. C4 crops
w=(102/255.0,102/255.0,0/255.0,255/255.0)## Dark yellow3 22.flooded trees
x=(46/255.0,139/255.0,87/255.0,255/255.0)## seagreen 23.flooded grassland
y=(255/255.0,0/255.0,0/255.0,255/255.0)## 24. red LCZ1
z=(255/255.0,30/255.0,0/255.0,255/255.0)## 25. red LCZ2
a1=(255/255.0,60/255.0,0/255.0,255/255.0)## 26. red LCZ3
b1=(255/255.0,90/255.0,0/255.0,255/255.0)## 27. red LCZ4
c1=(255/255.0,120/255.0,0/255.0,255/255.0)## 28. red LCZ5
d1=(255/255.0,150/255.0,0/255.0,255/255.0)## 29. red LCZ6
e1=(255/255.0,180/255.0,0/255.0,255/255.0)## 30. red LCZ7
f1=(255/255.0,210/255.0,0/255.0,255/255.0)## 31.red LCZ8
g1=(255/255.0,240/255.0,0/255.0,255/255.0)## 32. red LCZ9
h1=(128/255.0,128/255.0,128/255.0,255/255.0)## 33. gray LCZ10

In [ ]:
scheme=[a0,a,b,c,d,e,f,g,h,ii,k,l,m,n,o,p,q,r,s,t,u,v,w,x,y,z,a1,b1,c1,d1,e1,f1,g1,h1]
colSG = ListedColormap(scheme)
uSG=np.unique(DSG)
uniqnames=[]
schemeSG=[]
for u in uSG:
    if u==0:
        uniqnames.append('no data')
        schemeSG.append(scheme[0])
    else:
        indname=np.where(u==covernum)[0][0]
        uniqnames.append(covername[indname])
        schemeSG.append(scheme[indname])

# Plot data

In [ ]:
fig = plt.figure(figsize=(25, 25))
ax=fig.gca()
c=ax.pcolormesh(LONSG,LATSG,DSG,cmap=colSG, vmin=np.min(DSG)-1,vmax=np.max(DSG)+1)
sc=ax.scatter(LON,LAT,s=30,c='w')
cb=plt.colorbar(c, ticks=np.arange(np.min(DSG),np.max(DSG)+1),extendfrac='auto', spacing='proportional')
cb.set_ticks([i+1 for i in range(0, len(covername))])  
cb.set_ticklabels(covername)
cb.ax.tick_params(labelsize=20)
ax.set_xlim(np.min(LONSG),np.max(LONSG))
ax.set_ylim(np.min(LATSG),np.max(LATSG))
ax.set_xlabel('Longitude',fontsize=20)
ax.set_ylabel('Latitude',fontsize=20)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
fig.savefig('Coastline.png')

### ECOCLIMAP coastline is globally respecting Ireland's coastline well, the following is testing ECOCLIMAP coastline in details. Note that the coastline data surrounds the largest inland water bodies but not all, future test are required to test the inland water bodies and rivers representation in ECOCLIMAP.

## Statistics coastlines

### Cover coastline cover distribution

In [ ]:
ucoast=np.unique(covernum[Linenum]) # look for all unique cover number on the coastline
sizes=[len(np.where(u==covernum[Linenum])[0]) for u in ucoast] # get how many times this number appears
labels=[covername[np.where(u==covernum)[0][0]] for u in ucoast] # get the covername corresponding to the number

In [ ]:
threshold=5 # set up a % threshold to simplify the pie chart 
A=np.array(sizes)
S=np.sum(A)
R=100.0*A/S
indsave=np.where(R>5)[0]
indothers=np.where(R<=5)[0]

In [ ]:
Sothers=np.sum(np.array([sizes[io] for io in indothers]))
sizenew=[A[ids] for ids in indsave]
sizenew.append(Sothers)
labelsnew=[labels[ids] for ids in indsave]
labelsnew.append('Others')

In [ ]:
fig, ax = plt.subplots(figsize=(50,50), subplot_kw=dict(aspect="equal"))

indices=np.argsort(sizenew)
data = [sizenew[id] for id in indices]
ingredients = [labelsnew[id] for id in indices]

wedges, texts, autotexts = ax.pie(data, autopct='%1.1f%%')

leg=ax.legend(wedges, ingredients,
          title="Covers",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1),
          fontsize=30)
plt.setp(leg.get_title(),fontsize=30)
plt.setp(autotexts, size=30, weight="bold")


plt.show()
fig.savefig('pie_cover_coastline.png')

### 38.7% of coastline points are considered as sea and ocean by ECOCLIMAP. Are these points in a specific region? how far are these points from the ECOCLIMAP land values? On the other hand 32.2% are represented as temperate grassland how far are these points from the ECOCLIMAP sea?

### When coastline is represented by sea and ocean what is its distance from 'ECOCLIMAP' coastline

In [ ]:
soc=np.where(covernum[Linenum]==1)[0] #select sea and ocean coastline points
indx,indy=np.where(DSG!=1) # create all non-sea index
LATSGs=LATSG[indx]# create the corresponding latitude
LONSGs=LONSG[indy]# create the correponding longitude

In [ ]:
pointsx=[indicelat[s] for s in soc]
pointsy=[indicelon[s] for s in soc]

In [ ]:
idxs=[np.argmin(np.abs(LAT[px]-LATSGs)) for px in pointsx]
idys=[np.argmin(np.abs(LON[py]-LONSGs)) for py in pointsy]

In [ ]:
distances=[distance_latlon(LAT[pointsx[n]],LON[pointsy[n]],LATSGs[idxs[n]],LONSGs[idys[n]]) for n in range(0,len(idxs))]

In [ ]:
fig, ax = plt.subplots(figsize=(15,15))
ax.hist(distances)
ax.set_xlabel('Distance (m)',fontsize=20)
ax.set_ylabel('Occurence',fontsize=20)
ax.xaxis.set_tick_params(labelsize=20)
ax.yaxis.set_tick_params(labelsize=20)
fig.savefig('sea_point_histogram.png')

### Sea and ocean coastline index are less than 200m from the closest cover which is less than a pixel gap. 